In [2]:
from dotenv import load_dotenv
from openai import OpenAI
from typing import Tuple

import psycopg2
import os

import re
from typing import List

load_dotenv()

client = OpenAI()
def connect_to_db():
    """Connect to PostgreSQL database and return connection and cursor"""
    try:
        DATABASE_URL = os.getenv("DATABASE_URL")
        conn = psycopg2.connect(DATABASE_URL)
        cur = conn.cursor()
        return conn, cur
    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None, None

SYSTEM_PROMPT = """
تو یک تحلیل‌گر داده هستی که از جدول `final_true` برای پاسخ به سؤال‌های آماری استفاده می‌کنی.
وظیفه‌ات تولید کد postgres است که بر اساس `final_true` اطلاعات را تحلیل کند.

اطلاعات جدول مربوط به واردات و صادرات ایران از سال ۱۳۸۸ تا ۱۴۰۴ است و ستون‌های آن به شرح زیر است:
ستون "year" نوع "integer"، این ستون سال مبادله را مشخص میکند و عددی چهار رقمی بین ۱۳۸۸ تا ۱۴۰۴ است
ستون "month" نوع "integer"، این ستون ماه مبادله را مشخص میکند و عددی بین ۱ تا ۱۲ است
ستون "customs_name" نوع "text"،  این ستون اسم گمرکی که در آن مبادله رخ داده را مشخص می‌کند
ستون "country" نوع "text"،  این ستون اسم کشور که با آن مبادله کردیم را مشخص می‌کند
ستون "hs_code" نوع "text"،  کد تعرفه کالای مورد مبادله
ستون "weight" نوع "numeric"،  وزن کالای مورد مبادله به کیلوگرم
ستون "rial" نوع "numeric"،  ارزش کالای مورد مبادله به ریال
ستون "dollar" نوع "numeric"،  ارزش کالای مورد مبادله به دلار
ستون "type" نوع "text"،  نوع مبادله را مشخص می‌کند واردات یا صادرات

توجه:
- از میان ارزش دلار و ریال به صورت پیش فرض از دلار استفاده کن مگر اینکه صراحتاً در سؤال به ریال اشاره شده باشد.
- فقط کد postgres تولید کن و چیز اضافی نگو
- فقط کوئری SELECT بنویس که فقط از جدول final_true استفاده کند.
- از هیچ دستور DDL/DML مثل INSERT/UPDATE/DELETE/DROP/COPY/CREATE استفاده نکن.
- اگر نیاز به محدود کردن نتایج است از LIMIT استفاده کن.
- در کوئری‌هایی که نیاز به GROUP BY دارند، فقط روی ستون کلیدی (مثل hs_code یا year یا country) گروه‌بندی کن. 
- ستون‌های توضیحی (مثل customs_name) را در GROUP BY قرار نده مگر اینکه مستقیماً موضوع سؤال باشد.
- اگر لازم است توضیح یا متن یک کد (مثلاً customs_name) هم بیاید، آن را فقط به صورت یک ستون کمکی همراه با hs_code انتخاب کن (مثلاً با تابعی مثل MAX یا MIN).
- اگر شرط روی ستون customs_name یا country دقیقاً یک مقدار مشخص باشد (مثل «تهران» یا «ایران»)، فقط از مقایسه‌ی دقیق (= 'تهران') استفاده کن و از الگوهای کلی مثل ILIKE '%تهران%' استفاده نکن، مگر اینکه در سؤال صراحتاً اشاره شده باشد که همه‌ی موارد مشابه یا شامل آن واژه هم مدنظر هستند.

مثال:
ورودی:
تعداد کشور‌هایی که با ایران مبادله داشتند؟
خروجی:
SELECT COUNT(DISTINCT country) AS number_of_countries
FROM final_true;

"""


def question_to_query(question: str):
    response = client.chat.completions.create(
        model="gpt-5", 
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": question}
        ]
    )
    return response.choices[0].message.content


def execute_query(query: str, cur) -> Tuple[list, list]:
    """Execute PostgreSQL query and return results with column names"""
    
    cur.execute(query)
    columns = [desc[0] for desc in cur.description]
    results = cur.fetchall()
    return results, columns


def query_to_result(results: list, columns: list, user_question: str) -> str:
    """Convert DB results (all rows + column names) and user question to a Persian human-readable answer."""
    # Build a compact textual table representation
    # Convert all values to strings and escape newlines
    def fmt_cell(v):
        if v is None:
            return ""
        s = str(v)
        return s.replace("\n", " ")

    header = " | ".join(columns)
    rows_text = []
    for r in results:
        row_cells = [fmt_cell(c) for c in r]
        rows_text.append(" | ".join(row_cells))
    table_text = header + "\n" + ("\n".join(rows_text) if rows_text else "")

    SYSTEM_PROMPT_sql_to_text = """
                تو یک دستیار هوش مصنوعی هستی. وظیفه تو این است که سوال کاربر و نتیجه‌ای که از پایگاه داده استخراج شده را بگیری و یک پاسخ کوتاه و روان و قابل فهم به زبان فارسی تولید کنی.
            فقط و فقط پاسخ نهایی را برگردان.
            اگر لازم دیدی می‌توانی خلاصه‌ای از جدول (مثلاً تعداد ردیف‌ها، نام ستون‌های مهم یا چند ردیف اول) بیاوری، ولی کاربر ممکن است انتظار داشته باشد همه‌ی نتایج در پاسخ دیده شوند — بنابراین جدول کامل را نیز در صورت کوتاه بودن، به صورت مرتب و قابل خواندن وارد کن.
            توجه:
            - اعداد بسیار بزرگ را با کاما جدا کن (مثلاً ۱,۲۳۴,۵۶۷).
            - اگر عدد اعشاری است، آن را به دو رقم اعشار گرد کن (مثلاً ۱۲۳۴.۵۶).
            - چیزی برای ادامه گفتگو نپرس.
    """
    
    # Send to LLM for a natural Persian reply, providing the full table and the original question.
    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT_sql_to_text},
            {"role": "user", "content": f"سوال کاربر: '''{user_question}'''\n\nنتایج جدول (ستون‌ها و ردیف‌ها):\n\n{table_text}\n\nبا توجه به سوال کاربر و این داده‌ها، پاسخ فارسی، طبیعی و قابل فهم بنویس. فقط خروجی نهایی را بده."}
        ]
    )
    return response.choices[0].message.content




# Patterns
_SAFE_START = re.compile(
    r'^\s*(?:with\b[\s\S]*?select\b|select\b)',
    re.IGNORECASE
)

# کلیِ کلمات خطرناک که هر کدام در هر استیتمنت به‌عنوان کلمهٔ جداگانه ممنوع‌اند
_DANGEROUS_ANYWHERE = re.compile(
    r'\b('
    r'delete|drop|truncate|update|insert|alter|create|replace|'
    r'execute|exec|merge|call|grant|revoke|vacuum|analyze|'
    r'refresh\s+materialized\s+view|lock|cluster|listen|unlisten|notify|'
    r'do|security\s+definer|set|reset|begin|commit|rollback|savepoint|'
    r'pg_read_file|pg_ls_dir|pg_stat_file|lo_import|lo_export'
    r')\b',
    re.IGNORECASE
)

_PG_SLEEP = re.compile(r'\bpg_sleep\s*\(', re.IGNORECASE)
_COPY_PROGRAM = re.compile(r'\bcopy\b[\s\S]*\b(program|stdin|stdout)\b', re.IGNORECASE)


def _split_statements(sql: str) -> List[str]:
    """
    Split top-level SQL statements by semicolon, ignoring semicolons inside:
     - single quotes '...'
     - double quotes "..." (identifiers / strings)
     - dollar-quoted strings $$...$$ or $tag$...$tag$
    Returns list of trimmed statements (without trailing semicolons).
    """
    s = sql
    stmts = []
    cur = []
    i = 0
    n = len(s)

    # state
    in_sq = False   # single-quote '
    in_dq = False   # double-quote "
    in_dollar = False
    dollar_tag = None
    escape = False

    while i < n:
        ch = s[i]

        if in_sq:
            cur.append(ch)
            if ch == "'" and not escape:
                in_sq = False
            if ch == "\\" and not escape:
                escape = True
            else:
                escape = False
            i += 1
            continue

        if in_dq:
            cur.append(ch)
            if ch == '"' and not escape:
                in_dq = False
            if ch == "\\" and not escape:
                escape = True
            else:
                escape = False
            i += 1
            continue

        if in_dollar:
            cur.append(ch)
            # check for end tag
            if ch == '$' and s[i - len(dollar_tag) + 1:i + 1] == dollar_tag:
                # ensure we've matched the full tag ending (this is a quick check;
                # because we always append characters, it's safe)
                in_dollar = False
                dollar_tag = None
            i += 1
            continue

        # not inside any quote
        # detect start of single-quote
        if ch == "'":
            cur.append(ch)
            in_sq = True
            i += 1
            continue

        # double quote
        if ch == '"':
            cur.append(ch)
            in_dq = True
            i += 1
            continue

        # dollar-quote start? detect $tag$
        if ch == '$':
            # try to find a tag like $tag$
            m = re.match(r'\$[A-Za-z0-9_]*\$', s[i:])
            if m:
                tag = m.group(0)  # e.g. $$ or $abc$
                cur.append(tag)
                in_dollar = True
                dollar_tag = tag
                i += len(tag)
                continue
            else:
                # solitary dollar sign
                cur.append(ch)
                i += 1
                continue

        # semicolon at top-level => split
        if ch == ';':
            stmt = ''.join(cur).strip()
            if stmt:
                stmts.append(stmt)
            cur = []
            i += 1
            # skip possible spaces after semicolon (they'll be trimmed later)
            continue

        # otherwise just append
        cur.append(ch)
        i += 1

    # leftover
    last = ''.join(cur).strip()
    if last:
        stmts.append(last)
    return stmts


def validate_query(query: str, max_statements: int = 10) -> bool:
    """
    Validate that `query` consists of 1..max_statements top-level statements,
    each of which is a safe read-only SELECT (or WITH ... SELECT).
    Returns True if safe, False otherwise.
    """
    if not isinstance(query, str):
        return False

    q = query.strip()
    if not q:
        return False

    try:
        stmts = _split_statements(q)
    except Exception:
        # on any parser failure, be conservative and reject
        return False

    if not stmts:
        return False

    # limit number of statements to avoid DOS via huge number of small statements
    if len(stmts) > max_statements:
        return False

    for s in stmts:
        s_stripped = s.strip()

        # must start with SELECT or WITH ... SELECT
        if not _SAFE_START.match(s_stripped):
            return False

        # if WITH ... make sure the CTE definitions don't contain DML/DDL
        if re.match(r'^\s*with\b', s_stripped, re.IGNORECASE):
            # extract prefix before first SELECT (this will include the WITH ... definitions)
            prefix_before_first_select = re.split(r'\bselect\b', s_stripped, 1, flags=re.IGNORECASE)[0]
            if re.search(r'\b(insert|update|delete|merge|alter|create|drop|truncate|replace|execute|exec|call)\b',
                         prefix_before_first_select, re.IGNORECASE):
                return False

        # global dangerous keywords anywhere in the statement
        if _DANGEROUS_ANYWHERE.search(s_stripped):
            return False

        # specific dangerous patterns
        if _PG_SLEEP.search(s_stripped):
            return False

        if _COPY_PROGRAM.search(s_stripped):
            return False

    # all checks passed
    return True


def _make_like_pattern(value: str) -> str:
    # اگر کاربر خودش wildcard داده (% یا _) همون رو نگه دار
    if "%" in value or "_" in value:
        return value
    # بین کلمات % بگذار و دو سرش هم %
    words = value.strip().split()
    return "%" + "%".join(words) + "%"

def _extract_field_and_value(q: str):
    """
    به ترتیب این حالت‌ها رو چک می‌کنه:
    1) customs_name/country/country_name = '...'
    2) customs_name/country/country_name ILIKE '...'
    اگر پیدا کرد، (field, value) برمی‌گردونه، وگرنه None
    """
    # = '...'
    m = re.search(r"(customs_name|country|country_name)\s*=\s*'([^']+)'", q, flags=re.IGNORECASE)
    if m:
        return m.group(1), m.group(2)
    # ILIKE '...'
    m = re.search(r"(customs_name|country|country_name)\s*ILIKE\s*'([^']+)'", q, flags=re.IGNORECASE)
    if m:
        return m.group(1), m.group(2)
    return None

def suggest_like_matches(query: str, conn, limit: int = 100):
    """
    اگر در کوئری شرطی برای customs_name/country/country_name بود:
      SELECT DISTINCT <field> FROM final_true WHERE <field> ILIKE %s ORDER BY <field> LIMIT %s
    را اجرا می‌کند و خروجیِ مرتب‌شده را برمی‌گرداند (لیست از رشته‌ها).
    اگر چیزی پیدا نشد یا شرط نبود، None برمی‌گرداند.
    """
    found = _extract_field_and_value(query)
    if not found:
        return None

    field, value = found
    pattern = _make_like_pattern(value)

    sql = f"""
        SELECT DISTINCT {field}
        FROM final_true
        WHERE {field} ILIKE %s
        ORDER BY {field}
        LIMIT %s;
    """
    with conn.cursor() as cur:
        cur.execute(sql, (pattern, limit))
        rows = cur.fetchall()

    # rows مثل [('تهران',), ('غرب تهران',)...] → فقط عنصر اول هر تاپل را بردار
    options = [r[0] for r in rows]
    return options

def run_query_with_like(query: str, conn):
    """
    اگر شرط customs_name/country/country_name در کوئری بود، لیست مقادیر مشابه را برمی‌گرداند
    و متن پیشنهادی چاپ می‌کند. در غیر این صورت همان کوئریِ ورودی را (پس از تبدیل '=' به ILIKE)
    اجرا می‌کند و نتایج را برمی‌گرداند.
    """
    # اول تلاش برای پیشنهاد
    options = suggest_like_matches(query, conn)    
    return options

def talk_to_db(question: str, verbose: bool = False) -> str:
    conn = None
    cur = None
    chunks = []  # جمع‌آوری متن‌ها برای خروجی نهایی
    try:
        query = question_to_query(question)
        if not validate_query(query):
            print("این درخواست مجاز نیست. فقط کوئری‌های SELECT ساده مجاز هستند.\n" + query)

        if verbose:
            print("Generated Query:\n" + query + "\n")

        conn, cur = connect_to_db()
        if not cur or not conn:
            print("امکان اتصال به پایگاه داده وجود ندارد.")

        results, columns = execute_query(query, cur)
        if not results:
            print("هیچ نتیجه‌ای یافت نشد.")            
        
        # نمایش نتایج برای دیباگ
        if verbose:
            print(f"Query returned {len(results)} rows and {len(columns)} columns.\n")
            
            # عرض هر ستون رو حساب کنیم بر اساس بیشترین طول
            col_widths = [max(len(str(col)), max(len(str(row[i])) for row in results)) for i, col in enumerate(columns)]
            
            # سرستون‌ها
            header = " | ".join(col.ljust(col_widths[i]) for i, col in enumerate(columns))
            sep = "-+-".join("-" * col_widths[i] for i in range(len(columns)))
            
            # ردیف‌ها
            rows_str = "\n".join(
                " | ".join(str(row[i]).ljust(col_widths[i]) for i in range(len(columns)))
                for row in results
            )
            
            table = f"{header}\n{sep}\n{rows_str}"
            print(table + "\n")

        
        # جواب نهایی از LLM
        final_answer = query_to_result(results, columns, question)
        print(final_answer)

        # پیشنهادهای مشابه بر اساس customs/country
        options = run_query_with_like(query, conn)
        if options is not None and isinstance(options, list) and len(options) > 1:
            # متن کوتاه‌تر و شفاف‌تر:
            suggest_header = "در پایگاه داده این ها را نیز یافتیم، ممکن است مفید باشد و یا بخواید سوال خود را دقیق کنید"
            suggestion_lines = "\n".join(f"- {o}" for o in options)
            print(suggest_header + "\n" + suggestion_lines)

        

    except Exception as e:
        return f"خطا در اجرای درخواست: {str(e)}"
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()


# Done

# Done

In [12]:
print(talk_to_db("ایران با چند کشور در مبادله هست؟",verbose=True))

Generated Query:
SELECT COUNT(DISTINCT country) AS number_of_countries
FROM final_true;


طبق داده‌ها، ایران با ۲۰۰ کشور مبادله دارد.


In [13]:
print(talk_to_db("مجموع ارزش دلاری وارداتی از چین چقدر بوده است؟",verbose=True))

Generated Query:
SELECT SUM(dollar) AS total_import_value_usd_from_china
FROM final_true
WHERE type = 'واردات'
  AND country = 'چین';


مجموع ارزش دلاری وارداتی از چین برابر با ۱۸۷,۰۴۵,۸۲۴,۶۲۵.۸۲ دلار آمریکا بوده است.


In [15]:
print(talk_to_db("برترین گمرک ایران از نظر ارزش واردات کدام بوده است؟",verbose=True))

Generated Query:
SELECT
  customs_name,
  SUM(dollar) AS total_import_dollar
FROM final_true
WHERE type = 'واردات'
GROUP BY customs_name
ORDER BY total_import_dollar DESC
LIMIT 1;


برترین گمرک ایران از نظر ارزش واردات، گمرک شهید رجایی است؛ مجموع ارزش واردات آن برابر با ۱۶۷,۳۵۸,۸۷۹,۶۷۷.۱۲ دلار می‌باشد.


In [17]:
print(talk_to_db("برترین گمرک در مورد واردات از چین به لحاظ وزن کدام بوده؟",verbose=True))

columns: ['customs_name', 'total_weight']
row: ('شهیدرجایی', Decimal('20219035044.298500009171712'))
Generated Query:
SELECT customs_name, SUM(weight) AS total_weight
FROM final_true
WHERE type = 'واردات' AND country = 'چین'
GROUP BY customs_name
ORDER BY total_weight DESC
LIMIT 1;


Query returned 1 rows and 2 columns.


برترین گمرک برای واردات از چین از نظر وزن، گمرک شهید رجایی با مجموع وزن ۲۰,۲۱۹,۰۳۵,۰۴۴.۳۰ بوده است.


In [21]:
print(talk_to_db("کدام کالای وارداتی قیمت یک کیلوگرم واحدش از همه گرانتر بوده است؟",verbose=True))

Generated Query:
SELECT
  hs_code,
  SUM(dollar) / NULLIF(SUM(weight), 0) AS unit_price_usd_per_kg
FROM final_true
WHERE type = 'واردات'
  AND dollar > 0
  AND weight > 0
GROUP BY hs_code
ORDER BY unit_price_usd_per_kg DESC
LIMIT 1;


Query returned 1 rows and 2 columns.


hs_code  | unit_price_usd_per_kg
---------+----------------------
29329500 | 3756014.423076923077 


گران‌ترین کالای وارداتی کد HS ۲۹۳۲۹۵۰۰ است؛ قیمت هر کیلوگرم آن برابر با ۳,۷۵۶,۰۱۴.۴۲ دلار بوده است.


In [5]:
print(talk_to_db("کدام کالای وارداتی قیمت یک کیلوگرم واحدش از همه گرانتر بوده است؟",verbose=True))

Generated Query:
SELECT hs_code,
       SUM(dollar) / SUM(weight) AS dollar_per_kg
FROM final_true
WHERE type = 'واردات'
  AND weight > 0
  AND dollar IS NOT NULL
GROUP BY hs_code
ORDER BY dollar_per_kg DESC
LIMIT 1;


Columns: ['hs_code', 'dollar_per_kg']
Rows:
('29329500', Decimal('3756014.423076923077'))


کالای با کد HS 29329500 گران‌ترین قیمت هر کیلوگرم را دارد: 3,756,014.42 دلار آمریکا.


In [22]:
print(talk_to_db("چه کالایی را از تعداد گمرک بیشتری از عراق وارد کرده ایم؟",verbose=True))

Generated Query:
SELECT
  hs_code,
  COUNT(DISTINCT customs_name) AS customs_count
FROM final_true
WHERE type = 'واردات'
  AND country = 'عراق'
GROUP BY hs_code
ORDER BY customs_count DESC, hs_code
LIMIT 1;


Query returned 1 rows and 2 columns.


hs_code  | customs_count
---------+--------------
27101990 | 21           


کالای با کد تعرفه‌ی HS 27101990 از بیشترین تعداد گمرک‌های عراق وارد شده است — از ۲۱ گمرک.


In [8]:
print(talk_to_db("از لحاظ ارزش دلاری بیشترین واردات از چه کشوری بوده است؟",verbose=True))

Generated Query:
SELECT country, SUM(dollar) AS total_dollar_imports
FROM final_true
WHERE type = 'واردات'
GROUP BY country
ORDER BY total_dollar_imports DESC
LIMIT 1;


Columns: ['country', 'total_dollar_imports']
Rows:
('امارات متحده عربی', Decimal('213018046227.3747561602263190'))


بیشترین ارزش دلاری واردات از امارات متحده عربی بوده است، به میزان 213,018,046,227.37 دلار.


In [11]:
print(talk_to_db("کدام گمرک بیشترین وزن صادرات را داشته است؟",verbose=True))

Generated Query:
SELECT customs_name,
       SUM(weight) AS total_export_weight_kg
FROM final_true
WHERE type = 'صادرات'
GROUP BY customs_name
ORDER BY total_export_weight_kg DESC
LIMIT 1;


Columns: ['customs_name', 'total_export_weight_kg']
Rows:
('منطقه ویژه اقتصادی انرژی پارس', Decimal('274032079396.95'))


گمرک منطقه ویژه اقتصادی انرژی پارس بیشترین وزن صادرات را داشته است و مقدار آن 274,032,079,396.95 کیلوگرم است.


In [12]:
print(talk_to_db("در مجموع، ایران چه میزان کالا وارد کرده است؟ (به کیلوگرم و دلار)",verbose=True))

Generated Query:
SELECT
  SUM(weight) AS total_import_weight_kg,
  SUM(dollar) AS total_import_value_usd
FROM final_true
WHERE type = 'واردات';


Columns: ['total_import_weight_kg', 'total_import_value_usd']
Rows:
(Decimal('614953666691.043826836130983311'), Decimal('853724285558.9013404923803196'))


در مجموع، ایران 614,953,666,691.04 کیلوگرم کالا وارد کرده و ارزش آن 853,724,285,558.90 دلار است.


In [13]:
print(talk_to_db("صادرات به چین چه سهمی از کل صادرات ایران را دارد؟",verbose=True))

Generated Query:
SELECT ROUND(
  100.0 * SUM(CASE WHEN type = 'صادرات' AND country = 'چین' THEN dollar ELSE 0 END)
  / NULLIF(SUM(CASE WHEN type = 'صادرات' THEN dollar ELSE 0 END), 0)
, 2) AS export_share_to_china_percent
FROM final_true;


Columns: ['export_share_to_china_percent']
Rows:
(Decimal('22.10'),)


سهم صادرات ایران به چین از کل صادرات برابر با ۲۲٫۱۰ درصد است. یعنی حدوداً یک پنجم از کل صادرات ایران محسوب می‌شود.


In [16]:
print(talk_to_db("بیشترین ارزش ریالی واردات مربوط به چه کالایی بوده است؟",verbose=True))

Generated Query:
SELECT hs_code,
       SUM(rial) AS total_rial_import
FROM final_true
WHERE type = 'واردات'
GROUP BY hs_code
ORDER BY total_rial_import DESC
LIMIT 1;


Columns: ['hs_code', 'total_rial_import']
Rows:
('71081200', Decimal('3625466661963449.0'))


کالای با کد HS 71081200 بیشترین ارزش ریالی واردات را دارد و مقدار آن برابر با 3,625,466,661,963,449 ریال است.


In [17]:
print(talk_to_db("چه کشوری بیشترین تنوع کالایی را از ایران وارد کرده است؟",verbose=True))

Generated Query:
SELECT country, COUNT(DISTINCT hs_code) AS distinct_hs_codes
FROM final_true
WHERE type = 'صادرات'
GROUP BY country
ORDER BY distinct_hs_codes DESC, country
LIMIT 1;


Columns: ['country', 'distinct_hs_codes']
Rows:
('عراق', 6275)


کشور عراق با ۶٬۲۷۵ کد HS متفاوت بیشترین تنوع کالایی را از ایران وارد کرده است.


In [18]:
print(talk_to_db("چه کشوری بیشترین تنوع گمرکی را با ایران در صادرات دارد",verbose=True))

Generated Query:
SELECT country,
       COUNT(DISTINCT customs_name) AS customs_diversity
FROM final_true
WHERE type = 'صادرات'
GROUP BY country
ORDER BY customs_diversity DESC, country
LIMIT 1;


Columns: ['country', 'customs_diversity']
Rows:
('عراق', 170)


عراق با مقدار ۱۷۰، بیشترین تنوع گمرکی را با ایران در صادرات دارد.


In [19]:
print(talk_to_db("میانگین قیمت هر کیلو صادرات به کدام کشور کمتر است؟",verbose=True))

Generated Query:
SELECT country,
       SUM(dollar) / NULLIF(SUM(weight), 0) AS avg_price_per_kg_usd
FROM final_true
WHERE type = 'صادرات'
  AND weight > 0
  AND dollar IS NOT NULL
GROUP BY country
ORDER BY avg_price_per_kg_usd ASC
LIMIT 1;


Columns: ['country', 'avg_price_per_kg_usd']
Rows:
('هندوراس', Decimal('0.03178187749958613845'))


کمترین میانگین قیمت هر کیلو صادرات مربوط به کشور هندوراس است و مقدار آن حدود 0.0318 دلار آمریکا می‌باشد.


In [22]:
print(talk_to_db("کم‌ارزش‌ترین واردات ایران مربوط به کدام کالاست؟",verbose=True))

Generated Query:
SELECT hs_code, SUM(dollar) AS total_import_value_usd
FROM final_true
WHERE type = 'واردات' AND dollar IS NOT NULL
GROUP BY hs_code
ORDER BY total_import_value_usd ASC
LIMIT 1;


Columns: ['hs_code', 'total_import_value_usd']
Rows:
('29173920', Decimal('5.0'))


کم‌ارزش‌ترین واردات ایران در داده‌های شما مربوط به کد HS ۲۹۱۷۳۹۲۰ است و ارزش واردات آن برابر با ۵ دلار آمریکا بوده است.


In [23]:
print(talk_to_db("برترین گمرک از نظر وزن واردات از هند کدام است؟",verbose=True))

در پایگاه داده این ها را نیز یافتیم، ممکن است مفید باشد و یا بخواید سوال خود را دقیق کنید
- هند
- هندوراس
Generated Query:
SELECT customs_name, SUM(weight) AS total_import_weight
FROM final_true
WHERE type = 'واردات'
  AND country = 'هند'
GROUP BY customs_name
ORDER BY total_import_weight DESC
LIMIT 1;


Query returned 1 rows and 2 columns.


customs_name | total_import_weight       
-------------+---------------------------
شهیدرجایی    | 8973513761.215000001598193


برترین گمرک از نظر وزن واردات از هند، گمرک شهید رجایی است با مجموع وزن واردات برابر با ۸,۹۷۳,۵۱۳,۷۶۱.۲۲.

موارد مشابه در پایگاه داده یافت شد:
- هند
- هندوراس


In [14]:
print(talk_to_db("در گمرک بندرعباس بیشترین واردات مربوط به چه کالایی بوده است؟" ,verbose=True))

Generated Query:
SELECT hs_code, SUM(dollar) AS total_dollar
FROM final_true
WHERE type = 'واردات' AND customs_name = 'گمرک بندرعباس'
GROUP BY hs_code
ORDER BY total_dollar DESC
LIMIT 1;


هیچ نتیجه‌ای یافت نشد.


In [25]:
print(talk_to_db("کدام گمرک بیشترین تنوع کالای وارداتی را داشته است؟",verbose=True))

Generated Query:
SELECT customs_name,
       COUNT(DISTINCT hs_code) AS distinct_import_goods
FROM final_true
WHERE type = 'واردات'
GROUP BY customs_name
ORDER BY distinct_import_goods DESC
LIMIT 1;


Columns: ['customs_name', 'distinct_import_goods']
Rows:
('شهیدرجایی', 6852)


گمرک شهیدرجایی با بیشترین تنوع کالای وارداتی بوده است و ۶,۸۵۲ نوع کالا را شامل می‌شود.


In [15]:
print(talk_to_db("ارزش واردات گمرک بازرگان چقدر بوده است؟",verbose=True))

Generated Query:
SELECT SUM(dollar) AS total_import_value_usd
FROM final_true
WHERE type = 'واردات'
  AND customs_name = 'بازرگان';


Columns: ['total_import_value_usd']
Rows:
(Decimal('8588718566.5548999283261605'),)


ارزش کل واردات گمرک بازرگان 8,588,718,566.55 دلار آمریکا بوده است.
None


In [29]:
print(talk_to_db("از طریق چند گمرک مختلف کالا به روسیه صادر شده است؟",verbose=True))

Generated Query:
SELECT COUNT(DISTINCT customs_name) AS number_of_customs
FROM final_true
WHERE type = 'صادرات'
  AND country ILIKE '%روسیه%';


Columns: ['number_of_customs']
Rows:
(114,)


از طریق 114 گمرک مختلف کالا به روسیه صادر شده است.


In [16]:
print(talk_to_db("در تهران چه گمرک‌هایی داریم؟",verbose=True))

Generated Query:
SELECT DISTINCT customs_name AS tehran_customs
FROM final_true
WHERE customs_name = 'تهران'
ORDER BY tehran_customs;


Columns: ['tehran_customs']
Rows:
('تهران',)


در داده‌ها تنها یک گمرک در تهران ثبت شده: «تهران».
در پایگاه داده این ها را نیز یافتیم: ممکن است مفید باشد: و یا بخواید سوال خود را دقیق کنید
[('امور نمایشگاهی تهران',), ('تهران',), ('تهران امور واردات',), ('جنوب تهران',), ('غرب تهران',), ('معاونت حقوقی تهران',), ('معاونت صادرات تهران',), ('معاونت واردات و ترانزیت تهران',)]
None


In [17]:
print(talk_to_db("بیشترین ارزش واردات در گمرک تهران مربوط به چه کالایی بوده است؟",verbose=True))

Generated Query:
SELECT hs_code,
       SUM(dollar) AS total_import_value_usd
FROM final_true
WHERE type = 'واردات'
  AND customs_name = 'تهران'
GROUP BY hs_code
ORDER BY total_import_value_usd DESC NULLS LAST
LIMIT 1;


Columns: ['hs_code', 'total_import_value_usd']
Rows:
('87012000', Decimal('543975420.0'))


بیشترین ارزش واردات در گمرک تهران مربوط به کالای با کد تعرفه‌ای (HS) ۸۷۰۱۲۰۰۰ بوده است. ارزش این واردات برابر با ۵۴۳,۹۷۵,۴۲۰.۰۰ دلار آمریکا است. (نام کالا در داده‌های ارائه‌شده مشخص نشده است.)
None


In [3]:
print(talk_to_db("کدام گمرک بیشترین واردات کد تعرفه 84423000 را داشته است؟",verbose=True))

Generated Query:
SELECT customs_name,
       SUM(COALESCE(dollar, 0)) AS total_imports_dollar
FROM final_true
WHERE hs_code = '84423000'
  AND type = 'واردات'
GROUP BY customs_name
ORDER BY total_imports_dollar DESC
LIMIT 1;


Columns: ['customs_name', 'total_imports_dollar']
Rows:
('شهیدرجایی', Decimal('12513287.600000000001'))


گمرک شهیدرجایی بیشترین واردات کد تعرفه 84423000 را داشته است، به ارزش ۱۲,۵۱۳,۲۸۷.۶۰ دلار.


In [4]:
print(talk_to_db("کمترین واردات متعلق به کدام گمرک بوده است؟",verbose=True))

Generated Query:
SELECT customs_name,
       SUM(COALESCE(dollar, 0)) AS total_import_dollar
FROM final_true
WHERE type = 'واردات' AND customs_name IS NOT NULL
GROUP BY customs_name
ORDER BY total_import_dollar ASC, customs_name ASC
LIMIT 1;


Columns: ['customs_name', 'total_import_dollar']
Rows:
('بازارچه مرزی جالق ', Decimal('2525.83'))


کمترین واردات متعلق به گمرک بازارچه مرزی جالق است، با مقدار 2,525.83 دلار.


In [6]:
print(talk_to_db("۱۰ کالای گران که از امارات در ماه فروردین ۴۰۴ چه کالاهایی وارد شده‌اند؟",verbose=True))

Generated Query:
SELECT hs_code, SUM(COALESCE(dollar, 0)) AS total_dollar
FROM final_true
WHERE type = 'واردات'
  AND country = 'امارات'
  AND year = 1404
  AND month = 1
GROUP BY hs_code
ORDER BY total_dollar DESC
LIMIT 10;


هیچ نتیجه‌ای یافت نشد.


In [8]:
print(talk_to_db("میانگین قیمت هر کیلو صادرات به آلمان در شش ماه دوم ۴۰۳ چقدر بوده است؟",verbose=True))

Generated Query:
SELECT SUM(dollar) / NULLIF(SUM(weight), 0) AS avg_price_per_kg_usd
FROM final_true
WHERE type = 'صادرات'
  AND country = 'آلمان'
  AND year = 1403
  AND month BETWEEN 7 AND 12
  AND dollar IS NOT NULL
  AND weight > 0;


Columns: ['avg_price_per_kg_usd']
Rows:
(Decimal('5.4819728942773237'),)


میانگین قیمت هر کیلو صادرات به آلمان در شش ماهه دوم برابر با ۵.۴۸ دلار است.


In [9]:
print(talk_to_db("کدام کشور بیشترین صادرات  کد تعرفه 84423000 به ایران را از اسفند ۴۰۳ تا فروردین ۴۰۴ داشته؟",verbose=True))

Generated Query:
SELECT country, SUM(dollar) AS total_dollar
FROM final_true
WHERE hs_code = '84423000'
  AND type = 'واردات'
  AND (
    (year = 1403 AND month = 12) OR
    (year = 1404 AND month = 1)
  )
GROUP BY country
ORDER BY total_dollar DESC
LIMIT 1;


Columns: ['country', 'total_dollar']
Rows:
('آلمان', Decimal('1163540'))


کشور آلمان بیشترین صادرات کد تعرفه 84423000 به ایران را در بازه اسفند ۴۰۳ تا فروردین ۴۰۴ داشته است، به مبلغ ۱,۱۶۳,۵۴۰ دلار.


In [10]:
print(talk_to_db("۱۰ کالای مهم وارداتی از فرانسه چی هست؟",verbose=True))

Generated Query:
SELECT
  hs_code,
  SUM(dollar) AS total_dollar,
  SUM(weight) AS total_weight
FROM final_true
WHERE country = 'فرانسه'
  AND type = 'واردات'
GROUP BY hs_code
ORDER BY total_dollar DESC
LIMIT 10;


Columns: ['hs_code', 'total_dollar', 'total_weight']
Rows:
('98870338', Decimal('2320832279.96000000003'), Decimal('160330163.62'))
('98870314', Decimal('1345542089.2467301'), Decimal('120805220.735'))
('30049090', Decimal('699344245.57187750013'), Decimal('5655229.395999999994'))
('30043190', Decimal('376862518.45841'), Decimal('2458855.18'))
('98870313', Decimal('341611362.62816'), Decimal('35988889.0'))
('98870312', Decimal('278279734.09'), Decimal('23415956.64'))
('84099190', Decimal('246244418.48813170001'), Decimal('21067730.2400000000313'))
('98870316', Decimal('216088907.28'), Decimal('14316519.498'))
('90183190', Decimal('189276941.547172'), Decimal('2463162.98'))
('98870335', Decimal('175614439.01'), Decimal('12717917.0'))


بر اساس داده‌های ارائه‌شده، ۱۰ کالای مهم

In [11]:
print(talk_to_db("چه چیزی در هر ماه سال ۴۰۳ از سوریه وارد شده؟",verbose=True))

Generated Query:
SELECT
  month,
  hs_code,
  SUM(weight) AS total_weight_kg,
  SUM(dollar) AS total_dollar
FROM final_true
WHERE year = 1403
  AND country = 'سوریه'
  AND type = 'واردات'
GROUP BY month, hs_code
ORDER BY month, total_dollar DESC;


Columns: ['month', 'hs_code', 'total_weight_kg', 'total_dollar']
Rows:
(1, '25102000', Decimal('3000000'), Decimal('463235'))
(1, '85171390', Decimal('87'), Decimal('142769'))
(2, '25102000', Decimal('21001537'), Decimal('3053868'))
(2, '85171390', Decimal('236'), Decimal('397371'))
(3, '28100010', Decimal('126000'), Decimal('215522'))
(3, '25102000', Decimal('0'), Decimal('0'))
(3, '85171390', Decimal('0'), Decimal('0'))
(4, '25102000', Decimal('21230000'), Decimal('3294548'))
(4, '85171390', Decimal('45'), Decimal('123165'))
(4, '28100010', Decimal('50400'), Decimal('86406'))
(5, '25102000', Decimal('58000000'), Decimal('9069104'))
(5, '28100010', Decimal('0'), Decimal('0'))
(5, '85171390', Decimal('0'), Decimal('0'))
(6, '25102000', Decim

In [12]:
print(talk_to_db("واردات از ژاپن بیشتر در کدام حوزه‌هاست؟",verbose=True))

Generated Query:
SELECT
  hs_code,
  SUM(dollar) AS total_dollar,
  SUM(weight) AS total_weight,
  ROUND(100 * SUM(dollar) / NULLIF(SUM(SUM(dollar)) OVER (), 0), 2) AS share_percent
FROM final_true
WHERE type = 'واردات'
  AND country = 'ژاپن'
GROUP BY hs_code
ORDER BY total_dollar DESC
LIMIT 15;


Columns: ['hs_code', 'total_dollar', 'total_weight', 'share_percent']
Rows:
('98870443', Decimal('396312422.68'), Decimal('29897496.5710000000011'), Decimal('4.99'))
('84082090', Decimal('312735332.57262'), Decimal('23649997.03'), Decimal('3.94'))
('98870322', Decimal('291944263.39495'), Decimal('18626667.3900000005'), Decimal('3.68'))
('98870436', Decimal('289686138.42'), Decimal('17123811.23'), Decimal('3.65'))
('98870313', Decimal('221638937.73913'), Decimal('17882824.066'), Decimal('2.79'))
('72101200', Decimal('203672554.3079765'), Decimal('175462507.0'), Decimal('2.56'))
('72103000', Decimal('175879594.7800000003'), Decimal('148860586.0'), Decimal('2.21'))
('84295221', Decimal('16567369

In [13]:
print(talk_to_db("بیشترین حجم واردات ایران از عراق در سال ۴۰۴ مربوط به چه کالایی بوده؟",verbose=True))

Generated Query:
SELECT hs_code, SUM(weight) AS total_weight_kg
FROM final_true
WHERE type = 'واردات'
  AND country = 'عراق'
  AND year = 1404
GROUP BY hs_code
ORDER BY total_weight_kg DESC
LIMIT 1;


Columns: ['hs_code', 'total_weight_kg']
Rows:
('47079000', Decimal('13230471'))


در سال ۴۰۴ بیشترین حجم واردات ایران از عراق به کالای با کد تعرفه HS 47079000 اختصاص دارد و مقدار آن ۱۳,۲۳۰,۴۷۱ کیلوگرم است. این کد معمولاً مربوط به خمیر کاغذ شیمیایی است.


In [3]:
print(talk_to_db("تنوع کالاهای وارداتی از کره جنوبی چقدر است؟",verbose=True))

Generated Query:
SELECT COUNT(DISTINCT hs_code) AS import_goods_variety
FROM final_true
WHERE type = 'واردات'
  AND country = 'کره جنوبی';

Query returned 1 rows and 1 columns.

import_goods_variety
--------------------
4422                

تنوع کالاهای وارداتی از کره جنوبی برابر با ۴,۴۲۲ قلم کالا است.
None


In [17]:
print(talk_to_db("پنج گمرک برتر واردات از ترکیه در کدام گمرک‌ها انجام شده است؟",verbose=True))

Generated Query:
SELECT
  customs_name,
  SUM(dollar) AS total_import_value_usd
FROM final_true
WHERE type ILIKE '%واردات%'
  AND country ILIKE '%ترکیه%'
GROUP BY customs_name
ORDER BY total_import_value_usd DESC
LIMIT 5;


Columns: ['customs_name', 'total_import_value_usd']
Rows:
('بازرگان', Decimal('7243578938.1021327968664805'))
('جلفا', Decimal('5683589645.109168019304954'))
('منطقه ویژه اقتصادی بندر امام خمینی (ره)', Decimal('5509879002.0'))
('تهران', Decimal('5407682203.0'))
('معاونت واردات و ترانزیت تهران', Decimal('5159953533.209622660394809'))


این پنج گمرک بیشترین ارزش واردات از ترکیه را داشته‌اند:

- بازرگان — ۷,۲۴۳,۵۷۸,۹۳۸.۱۰ USD
- جلفا — ۵,۶۸۳,۵۸۹,۶۴۵.۱۱ USD
- منطقه ویژه اقتصادی بندر امام خمینی (ره) — ۵,۵۰۹,۸۷۹,۰۰۲.۰۰ USD
- تهران — ۵,۴۰۷,۶۸۲,۲۰۳.۰۰ USD
- معاونت واردات و ترانزیت تهران — ۵,۱۵۹,۹۵۳,۵۳۳.۲۱ USD


In [18]:
print(talk_to_db("ارزش دلاری واردات از سویس چقدر بوده است؟",verbose=True))

Generated Query:
SELECT SUM(dollar) AS total_imports_usd_from_switzerland
FROM final_true
WHERE type ILIKE 'وارد%'
  AND (
    country ILIKE '%سوئیس%' OR
    country ILIKE '%سوییس%' OR
    country ILIKE '%سویس%' OR
    country ILIKE '%سويس%' OR
    country ILIKE '%Switzer%'
  );


Columns: ['total_imports_usd_from_switzerland']
Rows:
(Decimal('29438038949.109690290377654'),)


ارزش دلاری واردات از سویس برابر با ۲۹,۴۳۸,۰۳۸,۹۴۹.۱۱ دلار است.


In [19]:
print(talk_to_db("بیشترین وزن واردات برای کدام کد تعرفه بوده است؟",verbose=True))

Generated Query:
SELECT hs_code,
       SUM(weight) AS total_weight_kg
FROM final_true
WHERE type = 'واردات'
GROUP BY hs_code
ORDER BY total_weight_kg DESC
LIMIT 1;


Columns: ['hs_code', 'total_weight_kg']
Rows:
('10059010', Decimal('96838058735.0'))


کد تعرفه ۱۰۰۵۹۰۱۰ بیشترین وزن واردات را دارد و مقدار آن ۹۶,۸۳۸,۰۵۸,۷۳۵.۰۰ کیلوگرم است.


In [20]:
print(talk_to_db("کدام کالا از بیشترین تعداد گمرک وارد شده است؟",verbose=True))

Generated Query:
SELECT hs_code,
       COUNT(DISTINCT customs_name) AS customs_count
FROM final_true
WHERE type ILIKE 'وارد%'
GROUP BY hs_code
ORDER BY customs_count DESC
LIMIT 1;


Columns: ['hs_code', 'customs_count']
Rows:
('39269099', 116)


کد کالای ۳۹۲۶۹۰۹۹ با ۱۱۶ گمرک بیشترین تعداد گمرک را دارد.


In [31]:
print(talk_to_db("کدام کالای وارداتی همزمان از چین و هند از فروردین ۴۰۳ تا فروردین ۴۰۴ به صورت مداوم به گمرک رجایی وارد شده؟",verbose=True))

Generated Query:
SELECT hs_code
FROM final_true
WHERE customs_name ILIKE '%رجایی%'
  AND type = 'واردات'
  AND (
    (year = 1403 AND month BETWEEN 1 AND 12) OR
    (year = 1404 AND month = 1)
  )
GROUP BY hs_code
HAVING COUNT(DISTINCT CASE WHEN country ILIKE '%چین%' THEN (year::text || '-' || LPAD(month::text, 2, '0')) END) = 13
   AND COUNT(DISTINCT CASE WHEN country ILIKE '%هند%' THEN (year::text || '-' || LPAD(month::text, 2, '0')) END) = 13
ORDER BY hs_code;


Columns: ['hs_code']
Rows:
('32041790',)
('38089113',)
('38089221',)
('38089223',)
('84833000',)


کالاهایی که به‌صورت مداوم و هم‌زمان از چین و هند از فروردین ۱۴۰۳ تا فروردین ۱۴۰۴ به گمرک رجایی وارد شده‌اند، دارای کدهای HS زیر هستند:

- 32041790
- 38089113
- 38089221
- 38089223
- 84833000

تعداد: ۵ مورد.

در صورت نیاز می‌توانم شرح این کدهای HS (نام کالاها) یا جزئیات بیشتری از واردات را هم ارائه دهم.


In [32]:
print(talk_to_db("واردات کد تعرفه 85171390 از کدام کشورها با قیمت مناسب تری انجام شده؟",verbose=True))

Generated Query:
SELECT
  country,
  ROUND(SUM(dollar)::numeric / NULLIF(SUM(weight), 0), 2) AS weighted_avg_price_per_kg_usd,
  SUM(weight) AS total_weight_kg,
  SUM(dollar) AS total_value_usd,
  COUNT(*) AS shipments_count
FROM final_true
WHERE type ILIKE '%وارد%'
  AND hs_code ILIKE '85171390%'
  AND weight > 0
  AND dollar IS NOT NULL
GROUP BY country
HAVING (SUM(dollar)::numeric / NULLIF(SUM(weight), 0)) < (
  SELECT SUM(dollar)::numeric / NULLIF(SUM(weight), 0)
  FROM final_true
  WHERE type ILIKE '%وارد%'
    AND hs_code ILIKE '85171390%'
    AND weight > 0
    AND dollar IS NOT NULL
)
ORDER BY weighted_avg_price_per_kg_usd ASC, total_weight_kg DESC;


Columns: ['country', 'weighted_avg_price_per_kg_usd', 'total_weight_kg', 'total_value_usd', 'shipments_count']
Rows:
('عراق', Decimal('277.68'), Decimal('1790'), Decimal('497053'), 1)
('فدراسیون روسیه', Decimal('392.38'), Decimal('144'), Decimal('56502'), 1)
('تایلند', Decimal('436.22'), Decimal('1094'), Decimal('477227'), 1)
('هن

In [33]:
print(talk_to_db("ماشین‌آلات صنعتی بیشتر از چه کشورهایی وارد شده‌اند؟",verbose=True))

Generated Query:
SELECT
  country,
  SUM(COALESCE(dollar, 0)) AS total_import_dollar
FROM final_true
WHERE type = 'واردات'
  AND LEFT(TRIM(hs_code), 2) = '84'
GROUP BY country
ORDER BY total_import_dollar DESC
LIMIT 10;


Columns: ['country', 'total_import_dollar']
Rows:
('چین', Decimal('41383417541.676147713416823'))
('امارات متحده عربی', Decimal('36661661398.9347662028042735'))
('آلمان', Decimal('14212983348.6019323612063921'))
('ایتالیا', Decimal('9583590419.0812784214378045'))
('ترکیه', Decimal('9484421771.246055455336256'))
('کره (جنوبی)', Decimal('7123277651.25260939241254'))
('بلژیک', Decimal('2004680572.383640787361445'))
('ژاپن', Decimal('1782334168.98424423523649'))
('هند', Decimal('1716662956.18225386517944'))
('فرانسه', Decimal('1543158297.541938191581144'))


بیشترین واردات ماشین‌آلات صنعتی از چین صورت گرفته است؛ رتبه‌های بعدی به ترتیب امارات متحده عربی و آلمان اختصاص دارد.

کشورها و ارزش کل واردات ماشین‌آلات صنعتی (دلار):
چین — ۴۱,۳۸۳,۴۱۷,۵۴۱.۶۸
امارات متحده عربی — ۳۶,۶۶۱

In [34]:
print(talk_to_db("واردات کد تعرفه 3003 از چه کشورهایی انجام شده است؟",verbose=True))

Generated Query:
SELECT DISTINCT country
FROM final_true
WHERE type ILIKE '%وارد%'
  AND hs_code LIKE '3003%'
ORDER BY country;


Columns: ['country']
Rows:
('^_^',)
('آلمان',)
('اتریش',)
('اروگوئه',)
('اسپانیا',)
('استرالیا',)
('امارات متحده عربی',)
('انگلستان (بریتانیا)',)
('ایالات متحده',)
('ایتالیا',)
('ایرلند',)
('برزیل',)
('بلژیک',)
('بلغارستان',)
('پرتغال',)
('ترکیه',)
('جمهوری چک',)
('چین',)
('دانمارک',)
('ژاپن',)
('سوئد',)
('سویس',)
('شیلی',)
('صربستان',)
('عمان',)
('فرانسه',)
('فنلاند',)
('قبرس',)
('قطر',)
('کانادا',)
('کره (جنوبی)',)
('کرواسی (Hrvatska)',)
('لبنان',)
('لهستان',)
('لوکزامبورگ',)
('لیتوانی',)
('مالزی',)
('مجارستان',)
('نیوزیلند (Aotearoa)',)
('هلند',)
('هند',)
('هنگ کنگ',)
('یونان',)


واردات با کد تعرفه 3003 از ۴۲ کشور انجام شده است. کشورها به‌شرح زیر هستند:

آلمان
اتریش
اروگوئه
اسپانیا
استرالیا
امارات متحده عربی
انگلستان (بریتانیا)
ایالات متحده
ایتالیا
ایرلند
برزیل
بلژیک
بلغارستان
پرتغال
ترکیه
جمهوری چک
چین
دانمارک
ژاپن
سوئد
سویس
شیلی
صربستان
عمان
فرانسه
فنل

In [35]:
print(talk_to_db("بیشترین ارزش واردات در فصل ۱۶ تا ۲۴ مربوط به چه کالایی است؟",verbose=True))

Generated Query:
SELECT
  hs_code,
  SUM(dollar) AS total_import_value_dollar
FROM final_true
WHERE
  type ILIKE '%وارد%'
  AND substring(hs_code FROM '^\s*([0-9]{2})')::int BETWEEN 16 AND 24
GROUP BY hs_code
ORDER BY total_import_value_dollar DESC
LIMIT 1;


Columns: ['hs_code', 'total_import_value_dollar']
Rows:
('23040000', Decimal('16463209325.75842600001'))


بیشترین ارزش واردات در فصل ۱۶ تا ۲۴ مربوط به کد HS 23040000 — کنجاله و سایر باقیمانده‌های جامد حاصل از استخراج روغن از دانه‌ها است. ارزش واردات: ۱۶,۴۶۳,۲۰۹,۳۲۵.۷۶ دلار آمریکا.


In [36]:
print(talk_to_db("کدام کالای وارداتی بالاترین وزن را داشته؟",verbose=True))

Generated Query:
SELECT hs_code,
       SUM(weight) AS total_weight_kg
FROM final_true
WHERE type ILIKE '%واردات%'
GROUP BY hs_code
ORDER BY total_weight_kg DESC NULLS LAST
LIMIT 1;


Columns: ['hs_code', 'total_weight_kg']
Rows:
('10059010', Decimal('96838058735.0'))


کالای وارداتی با کد تعرفه (HS) 10059010 بالاترین وزن را دارد: 96,838,058,735.00 کیلوگرم.


In [37]:
print(talk_to_db("میانگین قیمت واردات هر کیلو در کل سال 1404 چقدر بوده؟",verbose=True))

Generated Query:
SELECT SUM(dollar) / NULLIF(SUM(weight), 0) AS avg_import_price_per_kg_usd_1404
FROM final_true
WHERE year = 1404
  AND type ILIKE 'وارد%'
  AND weight > 0
  AND dollar IS NOT NULL;


Columns: ['avg_import_price_per_kg_usd_1404']
Rows:
(Decimal('1.5114816708864190'),)


میانگین قیمت واردات هر کیلو در سال ۱۴۰۴ برابر با ۱.۵۱ دلار آمریکا بوده است.


In [3]:
print(talk_to_db("از فروردین ۱۴۰۴ تا مرداد ۱۴۰۴ بیشترین نوسان قیمت هر کیلو مربوط به چه کالایی بوده؟",verbose=True))

Generated Query:
WITH monthly AS (
  SELECT
    hs_code,
    month,
    SUM(dollar) / NULLIF(SUM(weight), 0) AS unit_price_per_kg
  FROM final_true
  WHERE year = 1404
    AND month BETWEEN 1 AND 5
    AND dollar IS NOT NULL
    AND weight IS NOT NULL
    AND weight > 0
  GROUP BY hs_code, month
),
vol AS (
  SELECT
    hs_code,
    STDDEV_SAMP(unit_price_per_kg) AS price_per_kg_volatility
  FROM monthly
  GROUP BY hs_code
  HAVING COUNT(*) >= 2
)
SELECT hs_code, price_per_kg_volatility
FROM vol
ORDER BY price_per_kg_volatility DESC NULLS LAST
LIMIT 1;

Query returned 1 rows and 2 columns.

hs_code  | price_per_kg_volatility       
---------+-------------------------------
29371920 | 87014.037812997573545198352636

در بازهٔ فروردین تا مرداد ۱۴۰۴ بیشترین نوسان قیمت هر کیلو مربوط به کالای با کد HS 29371920 بوده است (مقدار نوسان: ۸۷,۰۱۴.۰۴).
None


In [39]:
print(talk_to_db("کالاهای ارزان‌قیمت (زیر ۱ دلار به ازای هر کیلو) کدام‌اند؟",verbose=True))

Generated Query:
SELECT
  hs_code,
  SUM(CASE WHEN dollar IS NOT NULL AND weight IS NOT NULL THEN dollar ELSE 0 END)
    / NULLIF(SUM(CASE WHEN dollar IS NOT NULL AND weight IS NOT NULL THEN weight ELSE 0 END), 0) AS dollar_per_kg,
  SUM(CASE WHEN dollar IS NOT NULL AND weight IS NOT NULL THEN weight ELSE 0 END) AS considered_weight_kg,
  SUM(CASE WHEN dollar IS NOT NULL AND weight IS NOT NULL THEN dollar ELSE 0 END) AS considered_dollar
FROM final_true
GROUP BY hs_code
HAVING
  SUM(CASE WHEN dollar IS NOT NULL AND weight IS NOT NULL THEN dollar ELSE 0 END)
    / NULLIF(SUM(CASE WHEN dollar IS NOT NULL AND weight IS NOT NULL THEN weight ELSE 0 END), 0) < 1
ORDER BY dollar_per_kg ASC;


Columns: ['hs_code', 'dollar_per_kg', 'considered_weight_kg', 'considered_dollar']
Rows:
('26180000', Decimal('0.00448974953846352065'), Decimal('379327481.0'), Decimal('1703085.3827462799'))
('25201000', Decimal('0.00841789163575916924'), Decimal('43293641795.94'), Decimal('364441185.1555969042820'))
('

In [46]:
print(talk_to_db("در چه مواردی ارزش ریالی واردات بسیار بالاتر از ارزش دلاری بوده؟ (اثر نرخ ارز یا تعرفه گمرکی)",verbose=True))

Generated Query:
WITH cases AS (
  SELECT
    year,
    hs_code,
    country,
    SUM(weight) AS total_weight_kg,
    SUM(dollar) AS total_dollar,
    SUM(rial) AS total_rial,
    SUM(rial) / NULLIF(SUM(dollar), 0) AS rial_per_usd
  FROM final_true
  WHERE type ILIKE '%وارد%'
  GROUP BY year, hs_code, country
),
annual_fx AS (
  SELECT
    year,
    SUM(rial) / NULLIF(SUM(dollar), 0) AS year_rial_per_usd
  FROM final_true
  WHERE type ILIKE '%وارد%'
  GROUP BY year
)
SELECT
  c.year,
  c.hs_code,
  c.country,
  c.total_weight_kg,
  c.total_dollar,
  c.total_rial,
  c.rial_per_usd,
  a.year_rial_per_usd AS avg_year_rial_per_usd,
  c.rial_per_usd / a.year_rial_per_usd AS times_above_year_avg
FROM cases c
JOIN annual_fx a USING (year)
WHERE c.total_dollar > 0
  AND a.year_rial_per_usd IS NOT NULL
  AND c.rial_per_usd / a.year_rial_per_usd >= 2
ORDER BY times_above_year_avg DESC, c.total_dollar DESC
LIMIT 100;


Columns: ['year', 'hs_code', 'country', 'total_weight_kg', 'total_dollar', 'to

In [47]:
print(talk_to_db("گمرک‌هایی که واردات گران‌قیمت‌تری داشته‌اند کدام‌اند؟",verbose=True))

Generated Query:
SELECT
  customs_name,
  SUM(dollar) AS total_import_value_usd,
  SUM(weight) AS total_weight_kg,
  SUM(dollar) / NULLIF(SUM(weight), 0) AS avg_import_price_usd_per_kg
FROM final_true
WHERE type ILIKE '%وارد%'
GROUP BY customs_name
HAVING SUM(weight) > 0 AND SUM(dollar) > 0
ORDER BY avg_import_price_usd_per_kg DESC;


Columns: ['customs_name', 'total_import_value_usd', 'total_weight_kg', 'avg_import_price_usd_per_kg']
Rows:
('فرودگاه اصفهان', Decimal('39858405'), Decimal('55598.0'), Decimal('716.9035756681895032'))
('منطقه آزاد شهر فرودگاهی امام خمینی (ره)', Decimal('6676184864.0'), Decimal('14002729'), Decimal('476.7774098891723178'))
('فرودگاه امام خمینی (ره)', Decimal('51326337818.733000004772'), Decimal('300380397.2400000000089'), Decimal('170.8711297086538203'))
('حوزه2 تجاری فرودگاه امام خمینی', Decimal('4928682285.860000011630397'), Decimal('31238384.3900000001043007'), Decimal('157.7764785888787764'))
('فرودگاه شیراز', Decimal('36239286.5836624'), Decimal('2739

In [50]:
print(talk_to_db("بیشترین واردات با نسبت قیمت به وزن بالا متعلق به چه کشور یا گمرکی است؟بیشترین واردات با نسبت قیمت به وزن بالا متعلق به چه کشور یا گمرکی است؟",verbose=True))

Generated Query:
WITH imports AS (
  SELECT *
  FROM final_true
  WHERE type ILIKE '%واردات%'
    AND dollar IS NOT NULL
    AND weight > 0
),
country_rank AS (
  SELECT
    country,
    SUM(dollar) AS total_dollar,
    SUM(weight) AS total_weight,
    SUM(dollar) / NULLIF(SUM(weight), 0) AS dollar_per_kg
  FROM imports
  GROUP BY country
  ORDER BY dollar_per_kg DESC NULLS LAST
  LIMIT 1
),
customs_rank AS (
  SELECT
    customs_name,
    SUM(dollar) AS total_dollar,
    SUM(weight) AS total_weight,
    SUM(dollar) / NULLIF(SUM(weight), 0) AS dollar_per_kg
  FROM imports
  GROUP BY customs_name
  ORDER BY dollar_per_kg DESC NULLS LAST
  LIMIT 1
)
SELECT 'country' AS dimension, country AS name, dollar_per_kg, total_dollar, total_weight
FROM country_rank
UNION ALL
SELECT 'customs' AS dimension, customs_name AS name, dollar_per_kg, total_dollar, total_weight
FROM customs_rank;


Columns: ['dimension', 'name', 'dollar_per_kg', 'total_dollar', 'total_weight']
Rows:
('country', 'تیمور', Dec

In [51]:
print(talk_to_db("واردات کد تعرفه 85171390 در گمرک تهران چه سهمی از کل واردات این کالا را دارد؟",verbose=True))

Generated Query:
SELECT
  ROUND(
    100.0 * SUM(CASE WHEN customs_name ILIKE '%تهران%' THEN COALESCE(dollar, 0) ELSE 0 END)
    / NULLIF(SUM(COALESCE(dollar, 0)), 0),
    2
  ) AS tehran_share_percent
FROM final_true
WHERE type ILIKE 'واردات%'
  AND TRIM(hs_code) = '85171390';


Columns: ['tehran_share_percent']
Rows:
(Decimal('0.01'),)


سهم گمرک تهران از واردات کد تعرفهٔ 85171390 معادل ۰.۰۱٪ از کل واردات این کالا است.


In [52]:
print(talk_to_db("مقایسه واردات کد تعرفه 3003 از هند و آلمان چگونه است؟",verbose=True))

Generated Query:
SELECT
  country_group AS country,
  SUM(dollar) AS total_dollar,
  SUM(weight) AS total_weight,
  COUNT(*) AS shipment_count,
  CASE WHEN SUM(weight) > 0 THEN SUM(dollar) / SUM(weight) END AS avg_dollar_per_kg
FROM (
  SELECT
    CASE
      WHEN country ILIKE '%هند%' THEN 'هند'
      WHEN country ILIKE '%آلمان%' THEN 'آلمان'
    END AS country_group,
    dollar,
    weight
  FROM final_true
  WHERE type = 'واردات'
    AND hs_code LIKE '3003%'
    AND (country ILIKE '%هند%' OR country ILIKE '%آلمان%')
) t
GROUP BY country_group
ORDER BY total_dollar DESC;


Columns: ['country', 'total_dollar', 'total_weight', 'shipment_count', 'avg_dollar_per_kg']
Rows:
('آلمان', Decimal('283744206.688'), Decimal('660768.9190000000003'), 62, Decimal('429.4151836279091085'))
('هند', Decimal('52712437.388693500044'), Decimal('336097.471830000002'), 71, Decimal('156.8367566161156617'))


خلاصه (تمام ردیف‌ها):
کشور | کل ارزش (دلار) | کل وزن (کیلوگرم) | تعداد محموله‌ها | میانگین دلار به ازا

In [53]:
print(talk_to_db("بیشترین واردات از چین در کدام گمرک انجام شده؟",verbose=True))

Generated Query:
SELECT
  customs_name,
  SUM(COALESCE(dollar, 0)) AS total_import_dollar
FROM final_true
WHERE type ILIKE 'وارد%'
  AND (country ILIKE '%چین%' OR country ILIKE '%چين%')
GROUP BY customs_name
ORDER BY total_import_dollar DESC
LIMIT 1;


Columns: ['customs_name', 'total_import_dollar']
Rows:
('منطقه ویژه اقتصادی شهید رجایی', Decimal('55496509652.428571429'))


بیشترین واردات از چین در گمرک منطقه ویژه اقتصادی شهید رجایی انجام شده است. مبلغ کل واردات: ۵۵,۴۹۶,۵۰۹,۶۵۲.۴۳ دلار.


In [54]:
print(talk_to_db("کالاهایی که هم در بندرعباس و هم در بوشهر وارد شده‌اند کدام‌اند؟",verbose=True))

Generated Query:
SELECT hs_code
FROM final_true
WHERE type = 'واردات'
  AND (customs_name ILIKE '%بندر%عباس%' OR customs_name ILIKE '%بوشهر%')
GROUP BY hs_code
HAVING COUNT(DISTINCT CASE WHEN customs_name ILIKE '%بندر%عباس%' THEN 1 END) >= 1
   AND COUNT(DISTINCT CASE WHEN customs_name ILIKE '%بوشهر%' THEN 1 END) >= 1
ORDER BY hs_code;


هیچ نتیجه‌ای یافت نشد.


In [55]:
print(talk_to_db("چه کالاهایی همزمان از کشورهای اروپایی و آسیایی وارد شده‌اند؟",verbose=True))

Generated Query:
WITH base AS (
  SELECT
    hs_code,
    CASE
      WHEN country ILIKE ANY (ARRAY[
        '%آلمان%', '%فرانسه%', '%ایتالیا%', '%اسپانیا%',
        '%انگلستان%', '%انگلیس%', '%بریتانیا%',
        '%اتریش%', '%هلند%', '%نیدرلند%', '%بلژیک%',
        '%سوئد%', '%نروژ%', '%دانمارک%', '%فنلاند%',
        '%ایرلند%', '%لهستان%', '%چک%', '%اسلوونی%', '%اسلواکی%',
        '%مجار%', '%یونان%', '%پرتغال%', '%رومانی%', '%بلغار%',
        '%کرواسی%', '%لتونی%', '%لیتوانی%', '%استونی%',
        '%لوکزامبورگ%', '%مالت%', '%ایسلند%',
        '%سوئیس%', '%سوییس%', '%اوکراین%',
        '%بوسنی%', '%صربستان%', '%آلبانی%', '%مقدونیه%', '%کوزوو%',
        '%مونته%نگرو%', '%مولدا%', '%بلاروس%', '%بیلاروس%',
        '%لیختن%', '%آندورا%', '%موناکو%', '%سن%مارینو%', '%واتیکان%'
      ]) THEN 'Europe'
      WHEN country ILIKE ANY (ARRAY[
        '%چین%', '%هند%', '%ژاپن%',
        '%کره%جنوبی%', '%کره جنوبی%', '%کره%شمالی%',
        '%امارات%', '%عربستان%', '%سعودی%', '%قطر%', '%کویت%', '%بح

In [56]:
print(talk_to_db("مقدار واردات کد تعرفه 10063000 از پاکستان در مهر ۴۰۳ چقدر هستش ؟",verbose=True))

Generated Query:
SELECT COALESCE(SUM(dollar), 0) AS total_imports_usd
FROM final_true
WHERE type = 'واردات'
  AND hs_code = '10063000'
  AND country ILIKE '%پاکستان%'
  AND year = 1403
  AND month = 7;


Columns: ['total_imports_usd']
Rows:
(Decimal('0'),)


بر اساس داده‌ها، مقدار واردات کد تعرفه 10063000 از پاکستان در مهر ۴۰۳ برابر ۰ دلار آمریکا است (هیچ وارداتی ثبت نشده).


In [57]:
print(talk_to_db("جمع کل ارزش واردات به دلار از پاکستان در پاییز ۴۰۳ چقدر بوده است",verbose=True))

Generated Query:
SELECT SUM(COALESCE(dollar, 0)) AS total_imports_usd_fall_1403
FROM final_true
WHERE type ILIKE '%واردات%'
  AND country ILIKE '%پاکستان%'
  AND year = 1403
  AND month IN (7, 8, 9);


Columns: ['total_imports_usd_fall_1403']
Rows:
(Decimal('186052381'),)


جمع کل ارزش واردات از پاکستان در پاییز ۱۴۰۳ برابر با ۱۸۶,۰۵۲,۳۸۱ دلار بوده است.


In [58]:
print(talk_to_db("جمع کل ارزش واردات به دلار از چین چقدر بوده است",verbose=True))

Generated Query:
SELECT
  SUM(COALESCE(dollar, 0)) AS total_imports_dollar_from_china
FROM final_true
WHERE type ILIKE '%وارد%'
  AND (
    country ILIKE '%چین%' OR
    country ILIKE '%china%'
  )
  AND country NOT ILIKE '%هنگ%کنگ%'
  AND country NOT ILIKE '%Hong%Kong%'
  AND country NOT ILIKE '%ماکائ%'
  AND country NOT ILIKE '%Macau%'
  AND country NOT ILIKE '%Macao%'
  AND country NOT ILIKE '%تایوان%'
  AND country NOT ILIKE '%Taiwan%'
  AND country NOT ILIKE '%تایپه%';


Columns: ['total_imports_dollar_from_china']
Rows:
(Decimal('187045824625.8180317527654721'),)


جمع کل ارزش واردات به دلار از چین ۱۸۷,۰۴۵,۸۲۴,۶۲۵.۸۲ دلار بوده است.


# Doing

In [59]:
print(talk_to_db("جمع کل ارزش واردات از چین چند میلیون دلار بوده است",verbose=True))

Generated Query:
SELECT COALESCE(SUM(dollar), 0) / 1000000.0 AS total_imports_from_china_million_usd
FROM final_true
WHERE type = 'واردات'
  AND (
    country ILIKE '%چین%' OR
    country ILIKE '%china%'
  )
  AND country NOT ILIKE '%تایپه%'
  AND country NOT ILIKE '%taipei%'
  AND country NOT ILIKE '%هنگ%کنگ%'
  AND country NOT ILIKE '%hong%kong%'
  AND country NOT ILIKE '%ماکائو%'
  AND country NOT ILIKE '%macau%';


Columns: ['total_imports_from_china_million_usd']
Rows:
(Decimal('187045.8246258180317528'),)


جمع کل ارزش واردات از چین ۱۸۷,۰۴۵.۸۲ میلیون دلار بوده است.


In [60]:
print(talk_to_db("پنج کشور برتر از لحاظ ارزش دلاری واردات",verbose=True))

Generated Query:
SELECT country,
       SUM(dollar) AS total_import_dollar
FROM final_true
WHERE type ILIKE '%واردات%'
GROUP BY country
ORDER BY total_import_dollar DESC
LIMIT 5;


Columns: ['country', 'total_import_dollar']
Rows:
('امارات متحده عربی', Decimal('213018046227.3747561602263190'))
('چین', Decimal('187045824625.8180317527654721'))
('ترکیه', Decimal('74610004655.2295914321978314'))
('آلمان', Decimal('43367504790.8297191167120803'))
('کره (جنوبی)', Decimal('42100215188.519063564876932'))


پنج کشور برتر از لحاظ ارزش دلاری واردات:
۱. امارات متحده عربی — ۲۱۳,۰۱۸,۰۴۶,۲۲۷.۳۷ دلار
۲. چین — ۱۸۷,۰۴۵,۸۲۴,۶۲۵.۸۲ دلار
۳. ترکیه — ۷۴,۶۱۰,۰۰۴,۶۵۵.۲۳ دلار
۴. آلمان — ۴۳,۳۶۷,۵۰۴,۷۹۰.۸۳ دلار
۵. کره (جنوبی) — ۴۲,۱۰۰,۲۱۵,۱۸۸.۵۲ دلار


In [61]:
print(talk_to_db("از نظر ارزش دلاری چه کالایی در صدر واردات به ایران قرار داشته است",verbose=True))

Generated Query:
SELECT
  hs_code,
  SUM(COALESCE(dollar, 0)) AS total_import_value_usd
FROM final_true
WHERE type ILIKE '%وارد%'
GROUP BY hs_code
ORDER BY total_import_value_usd DESC
LIMIT 1;


Columns: ['hs_code', 'total_import_value_usd']
Rows:
('10059010', Decimal('27748258039.194507'))


براساس این داده‌ها، کالای با کد HS 10059010 در صدر واردات به ایران قرار دارد. ارزش دلاری واردات این کالا: ۲۷,۷۴۸,۲۵۸,۰۳۹.۱۹ دلار.


In [62]:
print(talk_to_db("از نظر وزنی چه کالایی در صدر واردات به ایران قرار داشته است",verbose=True))

Generated Query:
SELECT hs_code, SUM(weight) AS total_weight_kg
FROM final_true
WHERE type ILIKE 'واردات%'
GROUP BY hs_code
ORDER BY total_weight_kg DESC
LIMIT 1;


Columns: ['hs_code', 'total_weight_kg']
Rows:
('10059010', Decimal('96838058735.0'))


از نظر وزنی، کالای با کد HS 10059010 در صدر واردات به ایران قرار دارد — مجموع وزن واردات: ۹۶,۸۳۸,۰۵۸,۷۳۵ کیلوگرم.


In [63]:
print(talk_to_db("10059010 این کد تعرفه بیشتر به کدام گمرک ایران وارد شده",verbose=True))

Generated Query:
SELECT customs_name, SUM(weight) AS total_weight_kg
FROM final_true
WHERE hs_code = '10059010'
  AND type ILIKE '%وارد%'
GROUP BY customs_name
ORDER BY total_weight_kg DESC
LIMIT 1;


Columns: ['customs_name', 'total_weight_kg']
Rows:
('بندر امام خمینی', Decimal('38868070247.0'))


کد تعرفه ۱۰۰۵۹۰۱۰ عمدتاً از گمرک بندر امام خمینی وارد شده است — مجموع وزن واردات: ۳۸,۸۶۸,۰۷۰,۲۴۷ کیلوگرم.


In [64]:
print(talk_to_db("10059010 این کد تعرفه بیشتر از کدام کشور به ایران وارد شده",verbose=True))

Generated Query:
SELECT country,
       SUM(dollar) AS total_dollar
FROM final_true
WHERE hs_code = '10059010'
  AND type ILIKE '%وارد%'
GROUP BY country
ORDER BY total_dollar DESC NULLS LAST
LIMIT 1;


Columns: ['country', 'total_dollar']
Rows:
('امارات متحده عربی', Decimal('8192876024.404067'))


برای کد تعرفه ۱۰۰۵۹۰۱۰، بیشترین واردات به ایران از امارات متحده عربی بوده است به ارزش ۸,۱۹۲,۸۷۶,۰۲۴.۴۰ دلار.


In [65]:
print(talk_to_db("این کد تعرفه رو 10059010 از منطقه ی ویژه اقتصادی امیرآباد چند دلار وارد کردیم ؟",verbose=True))

Generated Query:
SELECT SUM(dollar) AS total_dollar_imports_from_amirabad_sez
FROM final_true
WHERE hs_code = '10059010'
  AND type ILIKE 'وارد%'
  AND customs_name ILIKE '%منطقه%'
  AND customs_name ILIKE '%ویژه%'
  AND customs_name ILIKE '%اقتصاد%'
  AND customs_name ILIKE '%امیر%آباد%';


Columns: ['total_dollar_imports_from_amirabad_sez']
Rows:
(Decimal('1706439143.0'),)


برای کد تعرفهٔ 10059010 از منطقهٔ ویژهٔ اقتصادی امیرآباد، مجموع واردات برابر با ۱,۷۰۶,۴۳۹,۱۴۳.۰۰ دلار بوده است.


In [66]:
print(talk_to_db("از گمرک منطقه ویژه اقتصادی امیر آباد ۱۰ کالای برتر از نظر ارزش دلاری که وارد شده چه کالاهایی بودن",verbose=True))

Generated Query:
SELECT
  hs_code,
  SUM(COALESCE(dollar, 0)) AS total_dollar
FROM final_true
WHERE type ILIKE '%وارد%'
  AND customs_name ILIKE '%منطقه%ویژه%اقتصادی%'
  AND customs_name ILIKE '%امیر%آباد%'
GROUP BY hs_code
ORDER BY total_dollar DESC NULLS LAST
LIMIT 10;


Columns: ['hs_code', 'total_dollar']
Rows:
('15121100', Decimal('2173965994.0'))
('10059010', Decimal('1706439143.0'))
('10039000', Decimal('1309747888.0'))
('10019920', Decimal('223703107.0'))
('27131100', Decimal('140274535.0'))
('31042000', Decimal('115499057.0'))
('27131200', Decimal('54333277.0'))
('27040090', Decimal('43535685'))
('15071000', Decimal('40178383'))
('12019090', Decimal('36849614.0'))


ده کالای برتر وارداتی از گمرک منطقه ویژه اقتصادی امیرآباد بر اساس ارزش دلاری (کد HS و ارزش):

1) کد HS: 15121100 — ارزش دلاری: ۲,۱۷۳,۹۶۵,۹۹۴.۰۰  
2) کد HS: 10059010 — ارزش دلاری: ۱,۷۰۶,۴۳۹,۱۴۳.۰۰  
3) کد HS: 10039000 — ارزش دلاری: ۱,۳۰۹,۷۴۷,۸۸۸.۰۰  
4) کد HS: 10019920 — ارزش دلاری: ۲۲۳,۷۰۳,۱۰۷.۰۰  
5) کد HS: 271311

In [67]:
print(talk_to_db("۵ گمرک برتر ایران رو از نظر ارزش دلاری بهم بگو",verbose=True))

Generated Query:
SELECT
  customs_name,
  SUM(dollar) AS total_dollar_value
FROM final_true
GROUP BY customs_name
ORDER BY total_dollar_value DESC
LIMIT 5;


Columns: ['customs_name', 'total_dollar_value']
Rows:
('شهیدرجایی', Decimal('204795626134.610869276571732'))
('منطقه ویژه اقتصادی شهید رجایی', Decimal('182227666647.64929167904731'))
('منطقه ویژه اقتصادی انرژی پارس', Decimal('130923619988.4817384086508'))
('بندر امام خمینی', Decimal('90396542609.814344386322509'))
('منطقه ویژه اقتصادی بندر امام خمینی (ره)', Decimal('56012668733.931690476186'))


۵ گمرک برتر ایران از نظر ارزش دلاری:

۱. شهیدرجایی — ۲۰۴,۷۹۵,۶۲۶,۱۳۴.۶۱ دلار
۲. منطقه ویژه اقتصادی شهید رجایی — ۱۸۲,۲۲۷,۶۶۶,۶۴۷.۶۵ دلار
۳. منطقه ویژه اقتصادی انرژی پارس — ۱۳۰,۹۲۳,۶۱۹,۹۸۸.۴۸ دلار
۴. بندر امام خمینی — ۹۰,۳۹۶,۵۴۲,۶۰۹.۸۱ دلار
۵. منطقه ویژه اقتصادی بندر امام خمینی (ره) — ۵۶,۰۱۲,۶۶۸,۷۳۳.۹۳ دلار


In [68]:
print(talk_to_db("از گمرک منطقه ویژه اقتصادی شهید رجایی چه میزان کد تعرفه 10059010 وارد کردیم",verbose=True))

Generated Query:
SELECT SUM(dollar) AS total_import_value_usd
FROM final_true
WHERE hs_code = '10059010'
  AND type ILIKE '%وارد%'
  AND (
    customs_name ILIKE '%شهید%رجایی%'
    OR customs_name ILIKE '%شهيد%رجايي%'
  );


Columns: ['total_import_value_usd']
Rows:
(Decimal('301054102.90'),)


مجموع ارزش واردات کد تعرفه 10059010 از گمرک منطقه ویژه اقتصادی شهید رجایی برابر است با ۳۰۱,۰۵۴,۱۰۲.۹۰ دلار آمریکا.


In [69]:
print(talk_to_db("پنج گمرک برتر به لحاظ ارزش دلاری ،واردات کد تعرفه 71081200 چی بوده؟",verbose=True))

Generated Query:
SELECT customs_name,
       SUM(dollar) AS total_dollar
FROM final_true
WHERE type ILIKE '%واردات%'
  AND hs_code = '71081200'
GROUP BY customs_name
ORDER BY total_dollar DESC NULLS LAST
LIMIT 5;


Columns: ['customs_name', 'total_dollar']
Rows:
('منطقه آزاد شهر فرودگاهی امام خمینی (ره)', Decimal('6344487372'))
('فرودگاه امام خمینی (ره)', Decimal('4055967007'))
('حوزه2 تجاری فرودگاه امام خمینی', Decimal('796037002.28'))
('باشماق', Decimal('555447205'))
('تبریز', Decimal('262922316'))


پنج گمرک برتر به لحاظ ارزش دلاری واردات کد تعرفه 71081200 به‌ترتیب:

1. منطقه آزاد شهر فرودگاهی امام خمینی (ره): ۶,۳۴۴,۴۸۷,۳۷۲ دلار  
2. فرودگاه امام خمینی (ره): ۴,۰۵۵,۹۶۷,۰۰۷ دلار  
3. حوزه۲ تجاری فرودگاه امام خمینی: ۷۹۶,۰۳۷,۰۰۲.۲۸ دلار  
4. باشماق: ۵۵۵,۴۴۷,۲۰۵ دلار  
5. تبریز: ۲۶۲,۹۲۲,۳۱۶ دلار


In [70]:
print(talk_to_db("از چند گمرک واردات داشتیم",verbose=True))

Generated Query:
SELECT COUNT(DISTINCT customs_name) AS number_of_import_customs
FROM final_true
WHERE type ILIKE '%واردات%';


Columns: ['number_of_import_customs']
Rows:
(188,)


در مجموع از ۱۸۸ گمرک واردات داشتیم.


In [71]:
print(talk_to_db("جمع کل ارزش واردات به دلار چقدر بوده است",verbose=True))

Generated Query:
SELECT SUM(dollar) AS total_imports_dollar
FROM final_true
WHERE type ILIKE '%وارد%';


Columns: ['total_imports_dollar']
Rows:
(Decimal('853724285558.9013404923803196'),)


جمع کل ارزش واردات به دلار برابر است با ۸۵۳,۷۲۴,۲۸۵,۵۵۸.۹۰ دلار.


In [72]:
print(talk_to_db("چند کیلوگرم کد تعرفه 71081200 از عراق وارد کردیم؟",verbose=True))

Generated Query:
SELECT SUM(weight) AS total_weight_kg
FROM final_true
WHERE hs_code = '71081200'
  AND type ILIKE '%واردات%'
  AND country ILIKE '%عراق%';


Columns: ['total_weight_kg']
Rows:
(Decimal('9514'),)


مجموع وزن وارداتی از عراق برای کد تعرفه 71081200 برابر با ۹,۵۱۴ کیلوگرم است.
